In [24]:
# activate R magic
%load_ext rpy2.ipython
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
# from rpy2.robjects.conversion import localconverter

import pandas as pd
import os
import numpy as np
import time
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

data = pd.read_csv("/content/drive/My Drive/data/imputed_data.csv", parse_dates=[1], names=['Panel ID', 'Date', 'Category', 'Pack Size', 'Volume', 'Spend'], skiprows=1)
panel_demo = pd.read_excel("/content/drive/My Drive/data/DSA3101_Hackathon_Panelists_Demographics.xlsx")
panel_demo.columns = ["ID", "BMI", "Income", "Ethnicity", "Lifestage", "Stata", "HH", "Location"]

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [25]:
#### THIS SECTION IS FOR SELF DEFINED CLUSTERS ####

data['Category'] = data['Category'].map({'Baby Cereal': 'Baby',
                                         'Beer': 'Alcohol',
                                         'Belacan': 'Condiments',
                                         'Bird Nest': 'Health Supplements',
                                         'Biscuits': 'Snacks',
                                         'Bouilon': 'Soup',
                                         'Butter': 'Baking',
                                         'CSD': 'Drinks',
                                         'Cake': 'Baking',
                                         'Canned Product': 'Canned Product',
                                         'Cereal Beverage': 'Drinks',
                                         'Cereals': 'Snacks',
                                         'Cheese': 'Cultured Milk Products',
                                         'Chicken Essence': 'Health Supplements',
                                         'Choc/Nut Spread': 'Spread',
                                         'Chocolate': 'Snacks',
                                         'Coconut Milk': 'Condiments',
                                         'Coffee': 'Drinks',
                                         'Condensed/Evap Milk': 'Creamer',
                                         'Confectionery': 'Baking',
                                         'Cooking Oils': 'Cooking Oils',
                                         'Cooking Sauces': 'Condiments',
                                         'Cordials': 'Alcohol',
                                         'Creamer': 'Creamer',
                                         'Cultured Milk': 'Cultured Milk Products',
                                         'Drinking Water': 'Drinking Water',
                                         'Eggs': 'Eggs',
                                         'Energy Drinks': 'Drinks',
                                         'Flour': 'Baking',
                                         'Frozen Food': 'Frozen Food',
                                         'Fruit/Veg Juices': 'Drinks',
                                         'Ghee': 'Spread',
                                         'Honey': 'Spread',
                                         'Ice Cream': 'Cultured Milk Products',
                                         'Instant Noodles': 'Noodles',
                                         'Instant Soup': 'Soup',
                                         'Isotonic Drinks': 'Drinks',
                                         'Jam': 'Spread',
                                         'Kaya': 'Spread',
                                         'Liquid Milk': 'Drinks',
                                         'MSG': 'Condiments',
                                         'Margarine': 'Baking',
                                         'Milk Powder-Adult': 'Milk Powder-Adult',
                                         'Milk Powder-Infant': 'Baby',
                                         'Milk Powder-Kids': 'Baby',
                                         'Peanut Butter': 'Spread',
                                         'RTD Coffee': 'Drinks',
                                         'RTD Tea': 'Drinks',
                                         'Rice': 'Rice',
                                         'Salad Dressing': 'Condiments',
                                         'Savoury Spread': 'Spread',
                                         'Seasoning Powder': 'Condiments',
                                         'Snack': 'Snack',
                                         'Soy Milk': 'Drinks',
                                         'Spagetti': 'Noodles',
                                         'Spirits': 'Alcohol',
                                         'Sugar': 'Baking',
                                         'Tea': 'Drinks',
                                         'Tonic Food Drink': 'Drinks',
                                         'Wine': 'Alcohol',
                                         'Yoghurt Drink': 'Cultured Milk Products',
                                         'Yoghurts': 'Cultured Milk Products'})



In [26]:
data.sort_values("Date", inplace=True)

le = preprocessing.LabelEncoder()
data['Week'] = le.fit_transform(data.Date)

data = data[["Panel ID", "Category", "Spend", "Week"]] \
    .groupby(["Panel ID", "Category", "Week"]) \
    .agg({"Spend": sum}) \
    .reset_index() \
    .merge(data.groupby(["Panel ID"]).agg({"Week": min}).reset_index(), on="Panel ID")
data.columns = ["Panel ID", "Category", "Week", "Spend", "First Week"]
data.sort_values("Week", inplace=True)
data.tail()

,Panel ID,Category,Week,Spend,First Week
1444,Panel 101019101,Rice,155,13.0,0
441718,Panel 258016101,Eggs,155,8.4,0
577280,Panel 401045101,Baby,155,131.9,0
441530,Panel 258016101,Cooking Oils,155,7.5,0
508286,Panel 317052101,Drinks,155,35.5,0


In [27]:
# start from half a year late because we aren't sure if the customer is a new customer in the first 6 months
# end half a year early because of covid19 impacts
subset = data[(data["First Week"] >= 26) & (data["First Week"] < 156-26) & (data["Week"] >= 26) & (data["Week"] < 156-26)]
print(subset["Panel ID"].unique().shape)

(776,)


In [28]:
subset.head()

,Panel ID,Category,Week,Spend,First Week
851451,Panel 800792201,Creamer,26,4.4,26
851455,Panel 800792201,Drinking Water,26,12.0,26
849316,Panel 800751401,Drinks,26,34.2,26
849346,Panel 800751401,Eggs,26,11.0,26
849424,Panel 800751401,Snack,26,18.2,26


In [29]:
%%time
def week_EMA(x):
    lst = x.shift(-1)-x
    return lst.ewm(alpha=1e-3).mean().tail(1).values

def Monetary_EMA(x):
    return x.ewm(alpha=1e-3).mean().tail(1).values

def frequency(data):
    df = data.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index()

    df_last_week = data[["Panel ID"]].drop_duplicates()
    df_last_week["Week"] = 156 - 26 - 1

    df = df.merge(df_last_week, on=["Panel ID", "Week"], how="outer") \
        .fillna(0.) \
        .groupby("Panel ID") \
        .agg({"Week": [min, max, week_EMA]}) \
        .reset_index()
    
    df.columns = ["Panel ID", "First Week", "Last Week", "Frequency_EMA"]
    df = df[df["First Week"] <= 156 - 26 -3]    # remove customers that newly joined close to our cutoff date
    return df[["Panel ID", "Frequency_EMA"]]

def monetary(data):
    df = data.groupby(["Panel ID", "Week"]) \
        .agg({'Spend': sum}) \
        .reset_index() \
        .pivot_table(index=['Panel ID'], columns='Week', values='Spend') \
        .fillna(0.) \
        .reset_index() \
        .melt(id_vars=["Panel ID"], value_vars=list(range(26, 156-26)))

    df = df.merge(data[["Panel ID", "First Week"]], on="Panel ID", how="left")
    df = df[df["First Week"] <= df["Week"]]
    df = df.groupby("Panel ID") \
        .agg({"value": Monetary_EMA}) \
        .reset_index()

    df.columns = ["Panel ID", "Monetary_EMA"]
    return df

def first_purchases(df):
    pivot = data.pivot_table(index=['Panel ID', "First Week"], columns='Category', values='Spend')
    df = df[["Panel ID", "First Week"]] \
        .merge(data[["Panel ID", "Week", "Category", "Spend"]], how="left", left_on=["Panel ID", "First Week"], right_on=["Panel ID", "Week"]) \
        .pivot_table(index=['Panel ID', 'First Week'], columns='Category', values='Spend') \
        .reset_index()
    df = pd.concat([df, pd.DataFrame(columns=list(pivot.columns[~pivot.columns.isin(df.columns)]))]) \
        .fillna(0.) \
        .drop(['First Week'], axis=1)
    return df

subset_df = frequency(subset) \
    .merge(monetary(subset), on="Panel ID", how="left") \
    .merge(first_purchases(subset), on=["Panel ID"], how="left") \
    .merge(panel_demo, how="left", left_on="Panel ID", right_on="ID") \
    .drop(['ID'], axis=1)

print(subset_df.shape)

(762, 30)
CPU times: user 2.59 s, sys: 66.7 ms, total: 2.66 s
Wall time: 2.66 s


In [30]:
subset_df.head()

,Panel ID,Frequency_EMA,Monetary_EMA,Alcohol,Baby,Baking,Canned Product,Condiments,Cooking Oils,Creamer,Cultured Milk Products,Drinking Water,Drinks,Eggs,Frozen Food,Health Supplements,Milk Powder-Adult,Noodles,Rice,Snack,Snacks,Soup,Spread,BMI,Income,Ethnicity,Lifestage,Stata,HH,Location
0,Panel 122008104,1.274671,100.134157,0.0,17.9,9.9,0.0,11.7,5.0,4.8,0.0,3.6,48.7,19.0,17.0,0.0,23.6,11.0,0.0,2.0,17.9,0.0,0.0,Healthy,Income 3000 - 3999,North Malay,Teens and Toddlers,Rural,7+ Member HH,North
1,Panel 132002105,6.992997,2.567916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.7,0.0,0.0,0.0,0.0,0.0,38.8,0.0,0.0,0.0,0.0,Healthy,Income 1500 - 1999,North Chinese,Matured Families,Urban,1-3 Member HH,North
2,Panel 132025107,1.922439,19.617796,31.9,0.0,5.9,0.0,0.0,0.0,0.0,6.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,Healthy,Income >5000,North Chinese,Teens Aches,Urban,6 Member HH,North
3,Panel 204025103,1.091319,17.906255,0.0,0.0,0.0,0.0,3.9,18.8,0.0,6.0,0.0,0.0,0.0,11.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Under Weight,Income 4000 - 4999,Central Malay,Nesting Families,Urban,7+ Member HH,Central
4,Panel 211075102,21.780401,1.003899,0.0,0.0,3.8,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,2.2,0.0,0.0,0.0,Under Weight,Income >5000,Central Others,Teens Aches,Urban,4 Member HH,Central


In [31]:
subset_df.Frequency_EMA.value_counts(normalize=True, bins=range(0, 52, 2))
# ~53.8% of the panels in training have Frequency_EMA <= 2 so we can consider a customer to be frequent if the Frequency_EMA <= 2

(-0.001, 2.0]    0.538058
(2.0, 4.0]       0.154856
(4.0, 6.0]       0.086614
(6.0, 8.0]       0.035433
(10.0, 12.0]     0.031496
(8.0, 10.0]      0.020997
(16.0, 18.0]     0.017060
(12.0, 14.0]     0.017060
(24.0, 26.0]     0.017060
(20.0, 22.0]     0.014436
(22.0, 24.0]     0.007874
(32.0, 34.0]     0.007874
(14.0, 16.0]     0.007874
(28.0, 30.0]     0.006562
(18.0, 20.0]     0.006562
(30.0, 32.0]     0.003937
(46.0, 48.0]     0.002625
(40.0, 42.0]     0.002625
(26.0, 28.0]     0.001312
(34.0, 36.0]     0.001312
(36.0, 38.0]     0.001312
(42.0, 44.0]     0.001312
(44.0, 46.0]     0.001312
(48.0, 50.0]     0.001312
(38.0, 40.0]     0.000000
Name: Frequency_EMA, dtype: float64

In [32]:
subset_df.Monetary_EMA.value_counts(normalize=True, bins=range(0, int(subset_df.Monetary_EMA.max())+1, 40))
# ~79.5% of the panels in training have Monetary_EMA <= 40 so we can consider a customer to be low spender if the Monetary_EMA <= 40

(-0.001, 40.0]    0.795276
(40.0, 80.0]      0.158793
(80.0, 120.0]     0.032808
(120.0, 160.0]    0.007874
Name: Monetary_EMA, dtype: float64

# Modeling

In [33]:
scaler = None
def dataloader(df, scalar, target, scale=True):
    columns = list(panel_demo.columns)
    columns.remove("ID")
    data_df = df.copy()
    data_df["Frequency_EMA"] = (data_df["Frequency_EMA"] <= 2).map(int)
    data_df["Monetary_EMA"] = (data_df["Monetary_EMA"] > 40).map(int)
    data_df = pd.get_dummies(data_df, columns=columns, dtype=float)
    data = data_df.drop(['Panel ID', 'Frequency_EMA', 'Monetary_EMA'], axis=1)
    if scale:
        if scaler == None:
            X = pd.DataFrame(MinMaxScaler().fit_transform(data), 
                                columns=data.columns)
        else:
            X = pd.DataFrame(scaler.transform(data), 
                                columns=data.columns)
    else:
        X = data.copy()
    y = data_df[target]
    data = pd.concat([X, y], axis=1)
    return X, y, data, scalar
    
X, y, df, scaler = dataloader(subset_df, scaler, target="Frequency_EMA", scale=False)
# X, y, df, scaler = dataloader(subset_df, scaler, target="Monetary_EMA", scale=False)
print(X.shape, y.shape, df.shape)

(762, 59) (762,) (762, 60)


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify = y)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, train_df.shape, test_df.shape)

(571, 59) (191, 59) (571,) (191,) (571, 60) (191, 60)


In [35]:
%%time
%%R
library(tidyverse)
install.packages("pROC")
library(pROC)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/pROC_1.16.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 371822 bytes (363 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write 

CPU times: user 711 ms, sys: 31.6 ms, total: 743 ms
Wall time: 21.7 s


In [36]:
%%time
%%R -i X_train,X_test,y_train,y_test,train_df,test_df,X,y,df

full <- glm(Frequency_EMA~., data=df, family="binomial")
null <- glm(Frequency_EMA~1, data=df, family="binomial")
logreg <- step(null, scope=list(lower=formula(null), upper=formula(full)), direction="both", trace=0, k=2)    # AIC criterion
summary(logreg)


Call:
glm(formula = Frequency_EMA ~ Rice + BMI_Under.Weight + Ethnicity_East.Coast.Malay + 
    Cooking.Oils + Income_Income..5000 + Ethnicity_North.Chinese + 
    Lifestage_Matured.Families + Ethnicity_Central.Others + Ethnicity_Central.Chinese + 
    Condiments + Canned.Product + BMI_Over.Weight + Baking + 
    Cultured.Milk.Products + Lifestage_Empty.Nesters, family = "binomial", 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2403  -1.1238   0.5876   1.0173   2.2959  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 0.388917   0.141816   2.742 0.006099 ** 
Rice                       -0.017936   0.005609  -3.198 0.001385 ** 
BMI_Under.Weight           -0.978360   0.260892  -3.750 0.000177 ***
Ethnicity_East.Coast.Malay -0.345996   0.180965  -1.912 0.055882 .  
Cooking.Oils               -0.036853   0.012213  -3.017 0.002549 ** 
Income_Income..5000         0.437661   0.192401   2.27

In [37]:
%%R
prob_test <- predict(logreg, type="response")
acc_vec <- sapply(seq(0, 1, 0.001), function(x){
    class_test <- as.numeric(prob_test > x)
    accuracy <- mean(class_test == y)
})
threshold <- which.max(acc_vec)/1000
class_test <- as.numeric(prob_test > threshold)
CM <- table(class_test, y)
sensitivity <- CM[2,2]/(sum(CM[,2]))
specificity <- CM[1,1]/(sum(CM[,1]))
accuracy <- mean(class_test == y)
print(roc(class_test, y))
print(threshold)
print(accuracy)
CM

R[write to console]: Setting levels: control = 0, case = 1

R[write to console]: Setting direction: controls < cases




Call:
roc.default(response = class_test, predictor = y)

Data: y in 313 controls (class_test 0) < 449 cases (class_test 1).
Area under the curve: 0.6448
[1] 0.512
[1] 0.6469816
          y
class_test   0   1
         0 198 115
         1 154 295
